In [1]:
!apt-get update && apt install libpython3.7-dev

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
Reading package lists... Done
B

In [2]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
python-snappy is already the newest version (0.5-1.1build2).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [4]:
!pip install apache-beam

In [3]:
!pip install -q tensorflow_data_validation

In [5]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

In [6]:
from google.colab import files
uploaded = files.upload()

Saving pollution-small.csv to pollution-small.csv


In [7]:
dataset = pd.read_csv("pollution-small.csv")

In [8]:
dataset.head()

Date   pm10    no2    so2   soot
0  1/1/2009  98.67  14.10  44.38  34.81
1  1/2/2009  52.33  14.10  29.75  33.06
2  1/3/2009  74.67  20.50  36.25  39.25
3  1/4/2009  72.00  17.30  46.44  34.38
4  1/5/2009  81.00  25.64  56.56  45.59

In [9]:
dataset.shape

(2188, 5)

In [10]:
training_data = dataset[:1600]

In [11]:
training_data.describe()

pm10          no2          so2         soot
count  1600.000000  1600.000000  1600.000000  1600.000000
mean     49.656494    30.980519    16.229981    21.551956
std      35.211906    12.400788    10.621896    12.127354
min       6.380000     9.740000     4.010000     6.000000
25%      28.345000    22.567500     9.777500    14.400000
50%      38.835000    28.715000    13.275000    18.630000
75%      58.050000    36.370000    19.282500    24.072500
max     277.250000   138.010000   123.130000   107.650000

In [12]:
test_set = dataset[1600:]

In [13]:
test_set.describe()

pm10         no2         so2        soot
count  588.000000  588.000000  588.000000  588.000000
mean    44.648248   37.296922   13.605170   18.441310
std     28.992087   10.940050    5.098944    6.596459
min     11.900000   15.070000    4.990000    8.000000
25%     28.337500   29.217500   10.122500   14.410000
50%     35.555000   35.815000   12.345000   17.090000
75%     50.812500   43.872500   15.855000   20.962500
max    273.770000  106.030000   38.030000   87.210000

In [14]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe = training_data)

In [15]:
train_stats

datasets {
  num_examples: 1600
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 1600
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 

In [16]:
schema = tfdv.infer_schema(statistics = train_stats)

In [17]:
tfdv.display_schema(schema)

Type  Presence Valency Domain
Feature name                                
'Date'        BYTES  required              -
'pm10'        FLOAT  required              -
'no2'         FLOAT  required              -
'so2'         FLOAT  required              -
'soot'        FLOAT  required              -

In [18]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe = test_set)

In [19]:
anomalies = tfdv.validate_statistics(statistics = test_stats, schema = schema)

In [20]:
tfdv.display_anomalies(anomalies)

In [21]:
test_set_copy = test_set.copy()

In [22]:
test_set_copy.drop("soot", axis = 1, inplace = True)

In [23]:
test_set_copy.describe()

pm10         no2         so2
count  588.000000  588.000000  588.000000
mean    44.648248   37.296922   13.605170
std     28.992087   10.940050    5.098944
min     11.900000   15.070000    4.990000
25%     28.337500   29.217500   10.122500
50%     35.555000   35.815000   12.345000
75%     50.812500   43.872500   15.855000
max    273.770000  106.030000   38.030000

In [24]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe = test_set_copy)

In [25]:
anomalies_new = tfdv.validate_statistics(statistics = test_set_copy_stats, schema = schema)

In [26]:
tfdv.display_anomalies(anomalies_new)

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/display_util.py:186: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


Anomaly short description      Anomaly long description
Feature name                                                        
'soot'        Column dropped            Column is completely missing

In [27]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

In [28]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

In [29]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment = "SERVING")

In [30]:
tfdv.display_anomalies(serving_env_anomalies)

In [31]:
tfdv.write_schema_text(schema = schema, output_path = "pollution_schema.pbtxt")